In [94]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, AUC
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [95]:
df=pd.read_csv("./../data/ANN_Challenge_Train_v1.csv")
df=df.drop(0)
X = df.drop('Fault', axis=1)
y = df['Fault']
df.sample(5)

,Sensor_1,Sensor_2,Sensor_3,Sensor_4,Sensor_5,Sensor_6,Fault
1990,0.193988,-1.862589,0.547393,66.052032,0.462090,0.733931,0
1547,0.071719,-0.545922,-0.183726,146.173301,10.162535,0.029255,1
19,0.096365,0.674734,0.155185,66.959750,15.756295,0.282515,0
1504,0.230412,-0.143659,0.807455,68.153447,6.418759,0.294916,0
192,0.465272,0.905623,0.361826,92.687804,10.684811,0.430142,0


In [96]:
X.isna().sum()

Sensor_1      0
Sensor_2      0
Sensor_3      0
Sensor_4      0
Sensor_5    150
Sensor_6      0
dtype: int64

In [97]:
imputer = SimpleImputer(strategy='median')
X['Sensor_5'] = imputer.fit_transform(X[['Sensor_5']])

Sensor_5 column contained missing values
so I used median imputation for the missing values
I wanted a safe and statistically sound approach that doesn’t distort the original data distribution.

In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

it's crucial to normalize or scale features, especially when they vary in magnitude. 
In this step, I applied standardization to my dataset using StandardScaler

In [99]:
y.value_counts()

Fault
0    2399
1     600
Name: count, dtype: int64

there is imbalance in the class weight distribution.It can cause the model to become biased towards the majority class.
Rather than oversampling or undersampling (which can distort data or reduce information), I opted for class weighting, which is a safer and more flexible approach in neural networks.
I used sklearn.utils.class_weight.compute_class_weight() to automatically compute balanced weights.
Encourages the model to pay more attention to underrepresented faults.
Improves recall and F1-score for the minority class.

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

For the model Architecture, i tried many variation like with 3 layers etc. Dropout helped further in preventing overfitting. this model architecture was optimal for this dataset. i used Adam optimizer since that can computation faster. 

I used the EarlyStopping callback to monitor validation AUC during training. AUC is a powerful metric that measures how well the model separates the two classes. Since the dataset is imbalanced, AUC provides a more reliable picture than plain accuracy. Early stopping with patience=5 ensures training halts once validation performance plateaus, and restore_best_weights=True ensures the best-performing model (not just the final one) is retained.

In [101]:
cw_vals = class_weight.compute_class_weight('balanced', classes=np.array([0,1]), y=y_train)
class_weights = {0: cw_vals[0], 1: cw_vals[1]}


model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# 9. Callbacks for early stopping on validation AUC
early_stop = EarlyStopping(monitor='val_auc', mode='max', patience=5, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7358 - auc: 0.6887 - loss: 0.6476 - val_accuracy: 0.8396 - val_auc: 0.9033 - val_loss: 0.4471
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7847 - auc: 0.8460 - loss: 0.4983 - val_accuracy: 0.8333 - val_auc: 0.9052 - val_loss: 0.4116
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7970 - auc: 0.8499 - loss: 0.5009 - val_accuracy: 0.8562 - val_auc: 0.9097 - val_loss: 0.3912
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8266 - auc: 0.8760 - loss: 0.4558 - val_accuracy: 0.8521 - val_auc: 0.9127 - val_loss: 0.3817
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8119 - auc: 0.8611 - loss: 0.4768 - val_accuracy: 0.8667 - val_auc: 0.9150 - val_loss: 0.3642
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8263 - auc: 0.8693 - loss: 0.4669 - val_accuracy: 0.8625 - val_auc: 0.9158 - val_loss: 0.3765
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - 

In [102]:
results = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:   ", results[0])
print("Test Accuracy:", results[1])
print("Test AUC:    ", results[2])

y_prob = model.predict(X_test).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))


Test Loss:    0.37220999598503113
Test Accuracy: 0.875
Test AUC:     0.9008247256278992
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Classification Report:
               precision    recall  f1-score   support

           0     0.9617    0.8804    0.9193       485
           1     0.6282    0.8522    0.7232       115

    accuracy                         0.8750       600
   macro avg     0.7950    0.8663    0.8213       600
weighted avg     0.8978    0.8750    0.8817       600

Confusion Matrix:
 [[427  58]
 [ 17  98]]
ROC AUC: 0.901407440609592


In [103]:
import joblib
model.save('./../model/ann_model.keras')                 
joblib.dump(scaler, './../model/scaler.save')    
joblib.dump(imputer, './../model/imputer.save')


['./../model/imputer.save']